<a href="https://colab.research.google.com/github/anyaconda/myGraphML/blob/main/my_TigerGraph_FinCEN_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TigerGraph FinCEN Demo

To Get Started you will need an instance of TigerGraph running. The fastest way to get a box running is using https://tgcloud.io. If it's your first time using the cloud portal checkout [**Getting Started with TigerGrapoh 3.0**](https://www.tigergraph.com/blog/getting-started-with-tigergraph-3-0/)

>**Note:** This notebook will work if you have an existing solution, by creating a new graph, but it's suggested that you have a blank instance. At the bottom of the notebook you will see a commented out cell that will allow you to clear your existing solution. Use that command wisely as it deletes everything on your existing solution. 



## Setting Up Enviornment

### Grab and Import Packages

In [ ]:
# FETCH PACKAGES
!pip install pyTigerGraphBeta
!pip install dateparser
import pyTigerGraphBeta as tg
import dateparser
import pandas as pd

     |████████████████████████████████| 286kB 12.8MB/s 


### Setup Connection to TigerGraph

In [ ]:
# SETUP TIGERGRAPH CONNECTION
conn = tg.TigerGraphConnection(host="https://my-colab-fin-cen.i.tgcloud.io", username="tigergraph", password="tigergraph")

### Fetch Data From GitHub Repo

In [ ]:
# GRAB DATA FILES 
!git clone https://github.com/HerkTG/finsec.git

Cloning into 'finsec'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


### Loading DataFrames
This dataset is a subset of the [FinCEN Files](https://www.icij.org/investigations/fincen-files/explore-the-fincen-files-data/) data.


In [ ]:
bank_connections = pd.read_csv("/content/finsec/bank_connections.csv", sep=",")
transactions = pd.read_csv("/content/finsec/transactions_map.csv", sep=",")
country_stats = pd.read_csv("/content/finsec/country_data.csv", sep=",")

# CLEAN UP DATETIME FORMATING
# Cleaning up begin date in transactions dataframe
Ids = []
Values =[]
for items in transactions['begin_date'].iteritems():
     temp = list(items)
     try:
               temp[1] = str(dateparser.parse(temp[1]).date())
     except: 
               temp[1] = ""
     Ids.append(temp[0])
     Values.append(temp[1])

Res = pd.Series(Values, index=Ids)
transactions['begin_date'] = Res

# Cleaning up end date in transactions dataframe
Ids2 = []
Values2 =[]
for items2 in transactions['end_date'].iteritems():
     temp2 = list(items2)
     try:
               temp2[1] = str(dateparser.parse(temp2[1]).date())
     except: 
               temp2[1] = ""
     Ids2.append(temp2[0])
     Values2.append(temp2[1])

Res2 = pd.Series(Values2, index=Ids2)
transactions['end_date'] = Res2

# CLEAN TRANSACTIONS NA
transactions.fillna(float(0), inplace=True)

# CLEAN COUNTRY NA
country_stats.fillna(float(0), inplace=True)

In [ ]:
# TEST TRANSACTIONS DATAFRAME
transactions.head()

,id,icij_sar_id,filer_org_name_id,filer_org_name,begin_date,end_date,originator_bank_id,originator_bank,originator_bank_country,originator_iso,beneficiary_bank_id,beneficiary_bank,beneficiary_bank_country,beneficiary_iso,number_transactions,amount_transactions
0,223254,3297,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,2015-03-25,2015-09-25,cimb-bank-berhad,CIMB Bank Berhad,Singapore,SGP,barclays-bank-plc-london-england-gbr,Barclays Bank Plc,United Kingdom,GBR,68.0,5.689852e+07
1,223255,3297,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,2015-03-30,2015-09-25,cimb-bank-berhad,CIMB Bank Berhad,Singapore,SGP,barclays-bank-plc-london-england-gbr,Barclays Bank Plc,United Kingdom,GBR,118.0,1.162384e+08
2,223258,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,2012-07-05,2012-07-05,barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden...,Skandinaviska Enskilda Banken,Sweden,SWE,0.0,5.000000e+03
3,223259,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,2012-06-20,2012-06-20,barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden...,Skandinaviska Enskilda Banken,Sweden,SWE,0.0,9.990000e+03
4,223260,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,2012-05-31,2012-05-31,barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden...,Skandinaviska Enskilda Banken,Sweden,SWE,0.0,1.200000e+04


In [ ]:
# TEST BANK CONNECTIONS DATAFRAME
bank_connections.head()

,icij_sar_id,filer_org_name_id,filer_org_name,entity_b_id,entity_b,entity_b_country,entity_b_iso_code
0,4132,standard-chartered-plc,Standard Chartered Plc,habib-metropolitan-bank-limited-karachi-pakist...,Habib Metropolitan Bank Limited,Pakistan,PAK
1,3181,standard-chartered-plc,Standard Chartered Plc,standard-chartered-bank-dubai-uae-are,Standard Chartered Bank,United Arab Emirates,ARE
2,3181,standard-chartered-plc,Standard Chartered Plc,al-rostamani-international-exchange-uae-are,Al Rostamani International Exchange,United Arab Emirates,ARE
3,3181,standard-chartered-plc,Standard Chartered Plc,kuwait-turkish-participation-bank-inc-istanbul...,Kuwait Turkish Participation Bank Inc,Turkey,TUR
4,3181,standard-chartered-plc,Standard Chartered Plc,wall-street-exchange-center-llc-dubai-uae-are,Wall Street Exchange Center Llc,United Arab Emirates,ARE


In [ ]:
# TEST COUNTRY STATS DATAFRAME
country_stats.head()

,name,latitude,longitude,pop2019,GrowthRate,area,Density
0,Afghanistan,33.939110,67.709953,38041.754,1.0233,652230.0,58.3257
1,Albania,41.153332,20.168331,2880.917,0.9989,28748.0,100.2128
2,Algeria,28.033886,1.659626,43053.054,1.0185,2381741.0,18.0763
3,American Samoa,-14.270972,-170.132217,55.312,0.9978,199.0,277.9497
4,Andorra,42.546245,1.601554,77.142,1.0016,468.0,164.8333


## Create Graph

### Create Global Vertex and Edges

In [ ]:
# DEFINE / CREATE ALL EDGES AND VERTEX 
print(conn.gsql('''CREATE VERTEX Organization (PRIMARY_ID organization_id STRING , name STRING) WITH primary_id_as_attribute="true"''', options=[]))
print(conn.gsql('''CREATE VERTEX Filing (PRIMARY_ID filing_id UINT, idicij_sar_id UINT, transaction_count FLOAT, transaction_amount FLOAT) WITH primary_id_as_attribute="true"''', options=[]))
print(conn.gsql('''CREATE VERTEX Country (PRIMARY_ID name STRING , iso STRING, latitude DOUBLE, longitude DOUBLE, population FLOAT, growth_rate FLOAT, area FLOAT, density FLOAT) WITH primary_id_as_attribute="true"''', options=[]))
print(conn.gsql('''CREATE DIRECTED EDGE FILED (FROM Organization, TO Filing, begin_date STRING, end_date STRING) WITH REVERSE_EDGE="REVERSE_FILING"''', options=[]))
print(conn.gsql('''CREATE DIRECTED EDGE BENEFICIARY (FROM Filing, TO Organization) WITH REVERSE_EDGE="REVERSE_BENEFICIARY"''', options=[]))
print(conn.gsql('''CREATE DIRECTED EDGE ORIGINATOR (FROM Organization, TO Filing) WITH REVERSE_EDGE="REVERESE_ORIGINATOR"''', options=[]))
print(conn.gsql('''CREATE UNDIRECTED EDGE LOCATED (FROM Country, TO Organization)''', options=[]))


['The vertex type Organization is created.']
['The vertex type Filing is created.']
['The vertex type Country is created.']
['The edge type FILED is created.', 'The reverse edge type REVERSE_FILING is created.']
['The edge type BENEFICIARY is created.', 'The reverse edge type REVERSE_BENEFICIARY is created.']
['The edge type ORIGINATOR is created.', 'The reverse edge type REVERESE_ORIGINATOR is created.']
['The edge type LOCATED is created.']


### Create FinCEN Graph

In [ ]:
# CREATE GRAPH "fincen"
print(conn.gsql('''CREATE GRAPH fincen(Organization, Filing, Country, FILED, BENEFICIARY, ORIGINATOR, LOCATED, REVERSE_FILING, REVERSE_BENEFICIARY, REVERESE_ORIGINATOR)''', options=[]))

['Stopping GPE GSE RESTPP', 'Successfully stopped GPE GSE RESTPP in 12.615 seconds', 'Starting GPE GSE RESTPP', 'Successfully started GPE GSE RESTPP in 0.165 seconds', 'The graph fincen is created.']


In [ ]:
# TEST TO SEE IF GRAPH SCHEMA BUILT
print(conn.gsql('''LS''', options=[]))

['---- Graph fincen', 'Vertex Types:', '- VERTEX Organization(PRIMARY_ID organization_id STRING, name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"', '- VERTEX Filing(PRIMARY_ID filing_id UINT, idicij_sar_id UINT, transaction_count FLOAT, transaction_amount FLOAT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"', '- VERTEX Country(PRIMARY_ID name STRING, iso STRING, latitude DOUBLE, longitude DOUBLE, population FLOAT, growth_rate FLOAT, area FLOAT, density FLOAT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"', 'Edge Types:', '- DIRECTED EDGE FILED(FROM Organization, TO Filing, begin_date STRING, end_date STRING) WITH REVERSE_EDGE="REVERSE_FILING"', '- DIRECTED EDGE REVERSE_FILING(FROM Filing, TO Organization, begin_date STRING, end_date STRING) WITH REVERSE_EDGE="FILED"', '- DIRECTED EDGE BENEFICIARY(FROM Filing, TO Organization) WITH REVERSE_EDGE="REVERSE_BENEFICIARY"', '- DIRECTED EDGE REVERSE_BENEFICIARY(FROM Organ

### Set New Graph Parameters

In [ ]:
# SET NEW GRAPH PARAMS
conn.graphname = "fincen"
#conn.apiToken = conn.getToken(conn.createSecret()) -> fails -> manually generated secret
#src https://towardsdatascience.com/generating-a-secret-in-tigergraph-e5139d52dff6
conn.apiToken = conn.getToken("todo:secret")
#confirm 
#conn.apiToken

## Load Data

In [ ]:
# UPSERT VERTEX "Organization"
v_entity = conn.upsertVertexDataFrame(bank_connections, "Organization", "entity_b_id", attributes={"organization_id": "entity_b_id", "name": "entity_b"})
v_filer = conn.upsertVertexDataFrame(bank_connections, "Organization", "filer_org_name_id", attributes={"organization_id": "filer_org_name_id", "name": "filer_org_name"})
print(str(v_entity+v_filer) + " Organization VERTEX Upserted")

# UPSERT VERTEX "Filing"
v_filing = conn.upsertVertexDataFrame(transactions, "Filing", "id", attributes={"filing_id": "id", "idicij_sar_id": "icij_sar_id", "transaction_count": "number_transactions", "transaction_amount": "amount_transactions"})
print(str(v_filing) + " Filing VERTEX Upserted")

# UPSERT VERTEX "Country"
v_country = conn.upsertVertexDataFrame(country_stats, "Country", "name", attributes={"name": "name", "latitude": "latitude", "longitude": "longitude", "population": "pop2019", "growth_rate": "GrowthRate", "area": "area", "density": "Density"})
print(str(v_country) + " Country VERTEX Upserted")

# UPSERT EDGE "LOCATED"
e_origin_loc = conn.upsertEdgeDataFrame(transactions, "Country", "LOCATED", "Organization", from_id="originator_bank_country", to_id="originator_bank_id", attributes={})
e_benificiary_loc = conn.upsertEdgeDataFrame(transactions, "Country", "LOCATED", "Organization", from_id="beneficiary_bank_country", to_id="beneficiary_bank_id", attributes={})
print(str(e_origin_loc+e_benificiary_loc) + " LOCATED EDGES Upserted")

# UPSERT EDGE "BENEFICIARY"
e_benificiary = conn.upsertEdgeDataFrame(transactions, "Filing", "BENEFICIARY", "Organization", from_id="id", to_id="beneficiary_bank_id", attributes={})
print(str(e_benificiary) + " BENEFICIARY EDGES Upserted")

# UPSERT EDGE "ORIGINATOR"
e_originator = conn.upsertEdgeDataFrame(transactions, "Organization", "ORIGINATOR", "Filing", from_id="originator_bank_id", to_id="id", attributes={})
print(str(e_originator) + " ORIGINATOR EDGES Upserted")

# UPSERT EDGE "FILED"
e_filed = conn.upsertEdgeDataFrame(transactions, "Organization", "FILED", "Filing", from_id="filer_org_name_id", to_id="id", attributes={"begin_date": "begin_date", "end_date": "end_date"})
print(str(e_filed) + " ORIGINATOR EDGES Upserted")

# PRINT OUT STATS
print("=====TOTAL_UPSERTS=====")
print(str(v_entity+v_filer+v_filing+v_country) + " TOTAL VERTICES")
print(str(e_origin_loc+e_benificiary_loc+e_benificiary+e_originator+e_filed) + " TOTAL EDGES")


2212 Organization VERTEX Upserted
4507 Filing VERTEX Upserted
245 Country VERTEX Upserted
2365 LOCATED EDGES Upserted
4507 BENEFICIARY EDGES Upserted
4507 ORIGINATOR EDGES Upserted
4507 ORIGINATOR EDGES Upserted
=====TOTAL_UPSERTS=====
6964 TOTAL VERTICES
15886 TOTAL EDGES


## Extract Features for Machine Learning

In [ ]:
# RUN QUERY TO EXTRACT FEATURES
print(conn.runInterpretedQuery('''
INTERPRET QUERY () FOR GRAPH fincen { 
  SumAccum<FLOAT> @totalTransactionAmount;
  MaxAccum<FLOAT> @maxTransaction;
  MinAccum<FLOAT> @minTransaction;
  SumAccum<INT> @cntFiled;
  
  seed = {Organization.*};
  
  S1 = SELECT s 
       FROM seed:s-(FILED:e)->Filing:tgt
       ACCUM s.@cntFiled += 1,
             s.@totalTransactionAmount += tgt.transaction_amount,
             s.@maxTransaction += tgt.transaction_amount,
             s.@minTransaction += tgt.transaction_amount;

  PRINT S1[
            S1.@cntFiled AS Filing_Count,
            S1.@totalTransactionAmount AS Total_Transaction_Amount,
            S1.@maxTransaction AS Max_Transaction,
            S1.@minTransaction AS MinTransaction
          ]; 
}
'''))

[{'S1': [{'v_id': 'commerzbank-ag', 'v_type': 'Organization', 'attributes': {'Filing_Count': 3, 'Total_Transaction_Amount': 1400000, 'Max_Transaction': 650000, 'MinTransaction': 300000}}, {'v_id': 'deutsche-bank-ag', 'v_type': 'Organization', 'attributes': {'Filing_Count': 188, 'Total_Transaction_Amount': 6139140096, 'Max_Transaction': 802656000, 'MinTransaction': 755}}, {'v_id': 'the-western-union-company', 'v_type': 'Organization', 'attributes': {'Filing_Count': 3, 'Total_Transaction_Amount': 9402, 'Max_Transaction': 3134, 'MinTransaction': 3134}}, {'v_id': 'china-investment-corporation', 'v_type': 'Organization', 'attributes': {'Filing_Count': 103, 'Total_Transaction_Amount': 52275200, 'Max_Transaction': 16294911, 'MinTransaction': 140.44}}, {'v_id': 'valley-national-bank', 'v_type': 'Organization', 'attributes': {'Filing_Count': 11, 'Total_Transaction_Amount': 1107388.75, 'Max_Transaction': 500000, 'MinTransaction': 1750}}, {'v_id': 'banco-bilbao-vizcaya-argentaria-sa', 'v_type': '

# WARNING: DROP ALL - Will Delete everything in your graph!
Use this if you would like to start the notebook lab from the beginning.

In [ ]:
#print(conn.gsql('''DROP ALL''', options=[]))